In [1]:
import config as c 
from src.data import CombiDataset,make_loader,downsampling
from src.model import Model 
from src.freia_funcs import * 
from src.utils import * 
from tqdm import tqdm 
import tifffile  
import os 
import pandas as pd 
import torch.nn.functional as F 
import torch.nn as nn 
c.device = 'cuda:1'

my_experiment


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import resnet

encoder,bn = resnet.wide_resnet101_2()
encoder = encoder.to(c.device)

In [3]:
from torchvision.datasets import ImageFolder
from src.data import * 

def load_rgb_dataset(dataset_dir,class_name,train='train'):
    img_transform = transforms.Compose([transforms.ToTensor(),
                                        transforms.Resize((384)),
                                        transforms.Normalize(c.norm_mean,c.norm_std)])
    valid_img = (lambda x: 'rgb' in x and x.endswith('png'))
    if 'MVtecAD' in dataset_dir.split('/'):
        dataset = ImageFolder(os.path.join(dataset_dir,class_name,train),transform=img_transform)
    else:
        dataset = ImageFolder(os.path.join(dataset_dir,class_name,train),transform=img_transform,is_valid_file = valid_img)
    return dataset 

dataset = load_rgb_dataset(c.dataset_dir,'bottle',train='train')
loader = make_loader(dataset,shuffle=False)

In [11]:
for data in loader:
    img,label = to_device(data,c.device)
    with torch.no_grad():
        z = encoder(img)
    break 

unshuffle1 = nn.PixelUnshuffle(4)
unshuffle2 = nn.PixelUnshuffle(2)
z_0 =  unshuffle1(z[0])
z_1 = unshuffle2(z[1])
z_2 = z[2]
z = torch.concat([z_0,z_1,z_2],dim=1)


In [22]:
from src.model import * 

nf = get_nf(input_dim=7168).to(c.device)

In [23]:
nf(z).shape

torch.Size([8, 7168, 24, 24])